<a href="https://colab.research.google.com/github/MiraSharma8511/POC-Model-Training/blob/main/PoC_Pretrain_Deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets sentencepiece protobuf torch evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [ ]:
import json
import pandas as pd

with open("train_jobs_unique.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df["text"] = df["job"] + " - " + df["task"]
df.head()

,job,task,category,text
0,International Strategist,Processing statement applications,Augment with GenAI,International Strategist - Processing statemen...
1,"Merchandiser, Coach",Drafting occur reports,Human,"Merchandiser, Coach - Drafting occur reports"
2,"Designer, Technician",Managing discover processes,Augment with Automation,"Designer, Technician - Managing discover proce..."
3,Media Assembler,Drafting wife reports,Augment with Automation,Media Assembler - Drafting wife reports
4,Radio Strategist,Generating movie content,Augment with GenAI,Radio Strategist - Generating movie content


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["category"])

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

dataset = Dataset.from_pandas(df[["text", "label"]])
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small",use_auth_token=hf_token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small", num_labels=len(label_encoder.classes_)
)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    eval_strategy="no",
    save_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

/tmp/ipython-input-11-1133466255.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2020fc04300 (2020fc04300-bits-pilani) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.115400
1000,0.000100


TrainOutput(global_step=1000, training_loss=0.05774847981706262, metrics={'train_runtime': 859.8241, 'train_samples_per_second': 18.608, 'train_steps_per_second': 1.163, 'total_flos': 41883381573504.0, 'train_loss': 0.05774847981706262, 'epoch': 2.0})

In [ ]:
model.save_pretrained("job_category_model")
tokenizer.save_pretrained("job_category_model")

with open("label_encoder.json", "w") as f:
    json.dump(label_encoder.classes_.tolist(), f)


In [ ]:
# Load test input
with open("test_jobs_unique_input.json") as f:
    test_data = json.load(f)

texts = [item["job"] + " - " + item["task"] for item in test_data]

In [ ]:
# Load model/tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("job_category_model",use_auth_token=hf_token)
tokenizer = AutoTokenizer.from_pretrained("job_category_model",use_auth_token=hf_token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
tokens = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Predict
with torch.no_grad():
    outputs = model(**tokens)
    preds = torch.argmax(outputs.logits, dim=1).tolist()

In [ ]:
# Load label mapping
with open("label_encoder.json") as f:
    classes = json.load(f)

In [ ]:
# Add prediction to data
for i, item in enumerate(test_data):
    item["predicted_category"] = classes[preds[i]]

In [ ]:
# Save predictions
with open("predictions.json", "w") as f:
    json.dump(test_data, f, indent=2)

In [ ]:
# View predictions
test_data

[{'job': 'Medical officer',
  'task': 'Patient health diagnosis',
  'category': 'Human',
  'predicted_category': 'Augment with AI-ML'},
 {'job': 'PowerBI Analyst',
  'task': 'Dashboard creation',
  'category': 'Augment with AI-ML',
  'predicted_category': 'Augment with AI-ML'}]